# AdventureWorks - Försäljningsanalys

**Inledning**

Analysen undersöker följande frågor:
1. Hur många produkter finns i varje kategori?
2. Vilka produktkategorier genererar mest intäkter?
3. Hur har försäljningen utvecklats över tid?
4. Hur ser total försäljning och antal ordrar ut per år?
5. Vilka 10 produkter genererar mest försäljning?
6. Hur skiljer sig försäljningen mellan olika regioner, och hur många unika kunder har varje region?
7. Vilka regioner har högst/lägst genomsnittligt ordervärde, och skiljer det sig mellan individuella kunder och företagskunder?

Utöver dessa grundläggande analyser genomförs en djupanalys med fokus på regional försäljningsoptimering. Denna del undersöker vilka regioner som presterar bäst och sämst, hur produktkategoriernas försäljning skiljer sig mellan regioner samt om det finns säsongsmönster i efterfrågan. Analysen avslutas med datadrivna rekommendationer för hur försäljningen kan förbättras.

**Metod**

Data bearbetas med SQL och visualiseras och analyseras med Python.

**Avgränsningar**

Försäljning förekommer i flera valutor såsom kanadensisk dollar, australisk dollar, euro, brittiskt pund, tysk mark och fransk franc. Trots att en stor del av försäljningen sker i USA förekommer ingen försäljning uttryckt i amerikansk dollar i datan. Någon tydlig koppling mellan valutakoder och de belopp som används i kolumnerna LineTotal och TotalDue har inte kunnat identifieras.  

I denna analys har därför försäljningsbeloppen använts utan omräkning mellan valutor. Eftersom beloppen ligger inom ungefär samma storleksordning bedöms detta inte påverka jämförelser mellan produktkategorier, regioner eller tidsperioder i någon större utsträckning. Samtliga försäljningsbelopp redovisas därför utan angiven valutaenhet.  

**Sammanfattande insikter**

Analysen visar att företagets sortiment är fokuserat till ett fåtal kategorier där Bikes och Components är störst både till antal och försäljningsbelopp. Försäljningen har en uppåtgående trend över tid och uppvisar även ett säsongsmönster med en topp framförallt under sommarmånaderna.  

US Southwest är företagets starkaste marknad sett till total försäljning. Företagskunder har genomgående ett högre genomsnittligt ordervärde än individuella kunder, vilket är förväntat. Samtidigt framgår att US Central, US Northeast och US Southeast även har individuella kunder med relativt höga genomsnittliga ordervärden, vilket indikerar att privatkunder i dessa regioner i större utsträckning gör större inköp eller väljer dyrare produkter.

Kategorin Bikes dominerar försäljningen i alla regioner och uppvisar samma säsongsmönster med topp runt sommaren. Components är den näst största kategorin med tydlig koppling till regioner med hög cykelförsäljning.

Sammantaget visar analysen att försäljningsresultatet påverkas av en kombination av regional efterfrågan, kundtyp, säsong och produktfokus. Ett fortsatt fokus på cykelförsäljning, kompletterat med insatser för att öka försäljningen av Components samt en närmare analys av kundsegment med höga ordervärden, framstår som rimliga rekommendationer framåt.

**Källor**
Kursmaterial från Pythonprogrammering och statistisk dataanalys samt SQL.  
Heatmap: https://www.geeksforgeeks.org/python/how-to-draw-2d-heatmap-using-matplotlib-in-python/

In [ ]:
import pyodbc
import pandas as pd 
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
import matplotlib.pyplot as plt
import numpy as np 

In [ ]:
user = "SA"
password = quote_plus("!Kattluva2025")
server = "localhost:1433"
database = "AdventureWorks2025"
driver = quote_plus("ODBC Driver 18 for SQL server")

connection_string = (
    f"mssql+pyodbc://{user}:{password}@{server}/{database}"
    f"?driver={driver}&Encrypt=yes&TrustServerCertificate=yes"
)

engine = create_engine(connection_string)

try:
    with engine.connect():
        print("Anslutning till SQL Server lyckades")
except Exception as e:
    print("Kunde inte ansluta", e)

In [ ]:
def query_df(sql: str) -> pd.DataFrame: 
    with engine.connect() as conn:
        return pd.read_sql(text(sql), conn)

## Visualisering 1: Antal produkter per kategori
**Affärsfråga:** Hur många produkter finns i varje kategori?

**Tabeller som används**:  
Production.ProductCategory  
Production.ProductSubcategory  
Production.Product  

**Plan**:  
JOINa tabellerna  
Räkna DISTINCT produkter per kategori  
Skapa vertiklat stapeldiagram  
Analysera resultatet  

In [ ]:
query_vis1 = """
SELECT
    COALESCE(pc.Name, 'Saknar klassificering') AS Category,
    COUNT(DISTINCT p.ProductID) AS ProductCount
FROM Production.Product AS p
LEFT JOIN Production.ProductSubcategory AS ps
    ON p.ProductSubcategoryID = ps.ProductSubcategoryID
LEFT JOIN Production.ProductCategory AS pc
    ON ps.ProductCategoryID = pc.ProductCategoryID
GROUP BY COALESCE(pc.Name, 'Saknar klassificering')
ORDER BY ProductCount DESC;
"""
df_vis1 = query_df(query_vis1)

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(df_vis1["Category"], df_vis1["ProductCount"], color="lime", alpha=0.6)

for bar in bars:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 2.5,
        f"{int(height)}",
        va="center",
        ha="center"
    )

ax.set_title("Antal produkter per kategori", fontsize=14, fontweight="bold")
ax.set_xlabel("Kategori", fontsize=12)  
ax.set_ylabel("Antal produkter", fontsize=12)

plt .tight_layout()
plt.show()

### Insikter
Flest unika produkter finns i kategorin Components med 134 stycken vilket indikerar att företaget har ett fokus på komponenter och reservdelar.  
Den minsta kategorin är Accessories med 29 produkter. Detta kan vara ett utvecklingsområde.  

Det finns också ett stort antal unika produkter, 209 stycken, som saknar klassificering.  
_____________________________________________________________________________________________________________________________________________________

## Visualisering 2: Försäljning per produktkategori
**Affärsfråga:** Vilka produktkategorier genererar mest intäkter?

**Tabeller som används:**  
Production.ProductCategory  
Production.ProductSubcategory  
Production.Product  
Sales.SalesOrderDetail  

**Plan:**  
JOINa tabellerna  
Räkna total försäljning/intäkt  
Gruppera på kategori  
Sortera på total försäljning  
Skapa horisontellt stapeldiagram  
Analysera resultatet  

I denna analys används begreppen intäkt och försäljning synonymt och baseras på summan av LineTotal per produktkategori.  

In [ ]:
query_vis2 = """
SELECT 
    pc.Name AS Category,
    SUM(sod.LineTotal) AS TotalSales
FROM Production.ProductCategory AS pc
JOIN Production.ProductSubcategory AS psc
    ON pc.ProductCategoryID = psc.ProductCategoryID
JOIN Production.Product AS p
    ON psc.ProductSubcategoryID = p.ProductSubcategoryID
JOIN Sales.SalesOrderDetail AS sod
    ON p.ProductID = sod.ProductID 
GROUP BY pc.Name
ORDER BY TotalSales DESC;
"""
df_vis2 = query_df(query_vis2)
df_vis2 = df_vis2.sort_values("TotalSales", ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.barh(df_vis2["Category"], df_vis2["TotalSales"] / 1_000_000, color="magenta", alpha=0.6)

for bar in bars:
    width = bar.get_width()
    ax.text(
        width,
        bar.get_y() + bar.get_height()/2,
        f"{width:,.1f}",
        va="center",
        ha="left"
    )

ax.set_title("Total försäljning per kategori", fontsize=14, fontweight="bold")
ax.set_xlabel("Försäljning (miljoner)", fontsize=12)
ax.set_ylabel("Kategori", fontsize=12)

plt.tight_layout()
plt.show()

### Insikter
Den produktkategori som genererar mest intäkter är Bikes med cirka 94,7 miljoner medan minst intäkt genereras av kategorin Accessories med cirka 1,3 miljoner.    
_____________________________________________________________________________________________________________________________________________________

## Visualisering 3: Försäljningstrend över tid
**Affärsfråga:** Hur har försäljningen utvecklats över tid?  

**Tabeller som används:**  
Sales.SalesOrderHeader

**Plan:**  
Summera total försäljning per månad för åren med helårsdata, 2023 och 2024  
Plocka ut månad med lägst respektive högst försäljning  
Skapa linjediagram med trendlinje   
Gruppera månaderna för en djupare analys av säsongsmönster  
Skapa stapeldiagram  
Analysera resultatet  

In [ ]:
query_vis3 = """
SELECT 
    DATEFROMPARTS(YEAR(soh.OrderDate), MONTH(soh.OrderDate), 1) AS OrderMonth,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader AS soh
WHERE soh.OrderDate >= '2023-01-01' AND soh.OrderDate <  '2025-01-01'
GROUP BY DATEFROMPARTS(YEAR(soh.OrderDate), MONTH(soh.OrderDate), 1)
ORDER BY OrderMonth ASC;
"""
df_vis3 = query_df(query_vis3)

# Identifiera månader med lägst och högst försäljning
df_vis3["OrderMonth"] = pd.to_datetime(df_vis3["OrderMonth"], format="%Y-%m")
df_vis3["TotalSales"] = pd.to_numeric(df_vis3["TotalSales"])
min_row = df_vis3.loc[df_vis3["TotalSales"].idxmin()]
max_row = df_vis3.loc[df_vis3["TotalSales"].idxmax()]

print(
    f"Månad med lägst försäljning: {min_row['OrderMonth'].strftime('%Y-%m')}\n"
    f"Månad med högsta försäljning: {max_row['OrderMonth'].strftime('%Y-%m')}"
)

# Trendanalys
x = np.arange(len(df_vis3))
y = df_vis3["TotalSales"] / 1_000_000
coef = np.polyfit(x, y, 1)
trend = np.poly1d(coef)

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(df_vis3["OrderMonth"], df_vis3["TotalSales"] / 1_000_000, marker="o", linestyle="-", color="blue", alpha=0.6, label="Försäljning")
ax.plot(df_vis3["OrderMonth"], trend(x), linestyle="--", color="green", alpha=0.6, label="Trendlinje")

ax.set_title("Försäljningstrend per månad", fontsize=14, fontweight="bold")
ax.set_xlabel("Månad", fontsize=12)
ax.set_ylabel("Försäljning (miljoner)", fontsize=12)
ax.tick_params(axis="x", rotation=45)
ax.legend()

plt.tight_layout()
plt.show()

# Säsongsmönster
df_vis3["OrderMonth"] = pd.to_datetime(df_vis3["OrderMonth"])

df_vis3["Month"] = df_vis3["OrderMonth"].dt.month
seasonality = (
    df_vis3.groupby("Month")["TotalSales"]
      .mean()
      .reset_index()
)

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.bar(seasonality["Month"], seasonality["TotalSales"] / 1_000_000, color="orange", alpha=0.6)

ax.set_title("Genomsnittlig månatlig försäljning (säsongsmönster)", fontsize=14, fontweight="bold")
ax.set_xlabel("Månad", fontsize=12)
ax.set_ylabel("Genomsnittlig försäljning (miljoner)", fontsize=12)
ax.set_xticks(seasonality["Month"])

plt.tight_layout()
plt.show()

### Insikter
Försäljningen uppvisar en tydligt ökande trend över tid under perioden 2023–2024, vilket framgår av trendlinjen i det diagrammet Försäljningstrend per månad.  
Månaden med lägst försäljning är februari 2023, medan juni 2024 har högst försäljning under perioden.  

Analysen av genomsnittlig försäljning per kalendermånad visar även ett säsongsmönster, där försäljningen generellt är högre under sommarmånaderna, särskilt i juni,  
medan lägre nivåer återkommer under vissa vintermånader. Detta indikerar att försäljningen följer säsonger då det är bättre förutsättningar för att cykla och den relativst höga försäljningen i december kan vara kopplad till julhandeln.
_____________________________________________________________________________________________________________________________________________________

# Visualisering 4: Försäljning och antal ordrar per år
**Affärsfråga:** Hur ser total försäljning och antal ordrar ut per år?

**Tabeller som används:**  
Sales.SalesOrderHeader

**Plan:**  
Summera total försäljning  
Räkna antal ordrar  
Gruppera på år  
Sortera kronologiskt  
Skapa grupperat stapeldiagram 

In [ ]:
query_vis4 = """
SELECT 
    YEAR(soh.OrderDate) AS OrderYear,
    SUM(soh.TotalDue) AS TotalSales,
    COUNT(soh.SalesOrderID) AS OrderCount
FROM Sales.SalesOrderHeader AS soh
GROUP BY YEAR(soh.OrderDate)
ORDER BY OrderYear;
"""

df_vis4 = query_df(query_vis4) 

x = np.arange(len(df_vis4))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))

bars_sales = ax.bar(x - width/2, df_vis4["TotalSales"] / 1_000_000, width, label="Försäljning (miljoner)")
bars_orders = ax.bar(x + width/2, df_vis4["OrderCount"] / 1_000, width, label="Antal ordrar (tusental)")

for bar in bars_sales:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 0.5,
        f"{height:,.1f}",
        va="center",
        ha="center"
    )

for bar in bars_orders:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 0.5,
        f"{int(height)}",
        va="center",
        ha="center"
    )

ax.set_title("Försäljning och antal ordrar per år", fontsize=14, fontweight="bold")
ax.set_xlabel("År", fontsize=12)
ax.set_ylabel("Försäljning (miljoner) / Antal ordrar (tusental)", fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(df_vis4["OrderYear"])

ax.legend()
plt.tight_layout()
plt.show()

### Insikter
Diagrammet visar att både försäljning och antal ordrar ökar från 2022 till 2024, med 2024 som det hittills starkaste året sett till både total försäljning och antal ordrar.  
Under 2024 uppgår försäljningen till cirka 49 miljoner, samtidigt som antalet ordrar är som högst, cirka 14 tusen.

År 2022 har lägst försäljning och lägst antal ordrar, medan 2023 visar en tydlig ökning jämfört med föregående år.  
För 2025 ses en nedgång i både försäljning och antal ordrar, vilket sannolikt beror på att året inte representerar en fullständig försäljningsperiod.
_____________________________________________________________________________________________________________________________________________________

## Visualisering 5: Top 10 produkter
**Affärsfråga:** Vilka 10 produkter genererar mest försäljning?

**Tabeller som används:**  
Production.Product  
Sales.SalesOrderDetail  

**Plan:**  
JOINa tabellerna  
Summera total försäljning   
Gruppera på produkt   
Välja top 10  
Sortera på total försäljning  
Skapa horisontellt stapeldiagram  
Analysera resultatet

In [ ]:
query_vis5 = """
SELECT TOP 10
    p.Name AS ProductName,
    SUM(sod.LineTotal) AS TotalSales
FROM Production.Product AS p
JOIN Sales.SalesOrderDetail AS sod
    ON p.ProductID = sod.ProductID
GROUP BY p.Name
ORDER BY TotalSales DESC;
"""

df_vis5 = query_df(query_vis5)
df_vis5 = df_vis5.sort_values("TotalSales", ascending=True)

fig, ax = plt.subplots(figsize=(10, 6))

bars = ax.barh(df_vis5["ProductName"], df_vis5["TotalSales"] / 1_000_000, color="purple", alpha=0.6)

for bar in bars:
    width = bar.get_width()
    ax.text(
        width,
        bar.get_y() + bar.get_height()/2,
        f"{width:,.1f}",
        va="center",
        ha="left"
    )  

ax.set_title("Topp 10 produkter efter försäljning", fontsize=14, fontweight="bold")
ax.set_xlabel("Försäljning (miljoner)", fontsize=12)
ax.set_ylabel("Produktnamn", fontsize=12)

plt.tight_layout()
plt.show()

### Insikter
Analysen visar att Mountain-200-modellerna dominerar försäljningen, där Mountain-200 Black, 38 är den enskilda produkt som genererar mest försäljning med cirka 4,4 miljoner.  
Även övriga varianter inom Mountain-200-serien återfinns bland top-10, vilket indikerar att denna produktserie står för en betydande del av den totala försäljningen.

Bland de tio mest sålda produkterna finns även Road-modeller, men dessa genererar genomgående lägre försäljning jämfört med Mountain-200-produkterna.  
Den produkt med lägst försäljning inom topp-10-listan är Road-150 Red, 56, med cirka 1,8 miljoner i total försäljning.
_____________________________________________________________________________________________________________________________________________________

## Visualisering 6: Försäljning och antal kunder per region
**Affärsfråga:** Hur skiljer sig försäljningen mellan olika regioner, och hur många unika kunder har varje region?

**Tabeller som används:**  
Sales.SalesTerritory  
Sales.SalesOrderHeader  
Sales.Customer

**Plan:** 
JOINa tabellerna  
Summera total försäljning  
Räkna antal kunder  
Gruppera på region   
Sortera på total försäljning  
Skapa grupperat stapeldiagram  
Analysera resultatet

In [ ]:
query_vis6 = """
SELECT
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END AS Region,
    COUNT(DISTINCT c.CustomerID) AS UniqueCustomers,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesTerritory AS st
LEFT JOIN Sales.SalesOrderHeader AS soh
    ON st.TerritoryID = soh.TerritoryID
LEFT JOIN Sales.Customer AS c
    ON soh.CustomerID = c.CustomerID
GROUP BY
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END
ORDER BY TotalSales DESC;
"""

df_vis6 = query_df(query_vis6)

x = np.arange(len(df_vis6))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars_sales = ax.bar(x - width/2, df_vis6["TotalSales"] / 1_000, width, label="Försäljning (tusental)")
bars_customers = ax.bar(x + width/2, df_vis6["UniqueCustomers"], width, label="Antal kunder")

for bar in bars_sales:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 500,
        f"{height:,.0f}",
        va="center",
        ha="center"
    )

for bar in bars_customers:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 500,
        f"{int(height)}",
        va="center",
        ha="center"
    )

ax.set_title("Försäljning och antal kunder per region", fontsize=14, fontweight="bold")
ax.set_xlabel("Region", fontsize=12)
ax.set_ylabel("Försäljning (tusental) / Antal kunder", fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(df_vis6["Region"], rotation=45)
ax.legend()

plt.tight_layout()
plt.show()

### Insikter
Analysen visar att US Southwest är den region som genererar högst försäljning, med cirka 27 miljoner, och även har flest kunder, omkring 4,5 tusen.  
Därefter följer Canada och US Northwest, som båda har relativt hög försäljning men med färre kunder jämfört med US Southwest.

Germany och US Northeast tillhör de regioner som har lägst försäljning och färre kunder.  

Sammantaget indikerar detta att både kundbasens storlek och regional efterfrågan bidrar till skillnaderna i försäljning mellan regionerna.
_____________________________________________________________________________________________________________________________________________________

## Visualisering 7: Genomsnittligt ordervärde per region och kundtyp
**Affärsfråga:** Vilka regioner har högst/lägst genomsnittligt ordervärde, och skiljer det sig mellan individuella kunder och företagskunder?

**Tabeller som används:**  
Sales.SalesTerritory  
Sales.SalesOrderHeader  
Sales.Customer  
Sales.Store

**Plan:**  
JOINa tabellerna  
Summera försäljning för individuella kunder respektive företagskunder  
Räkna antal ordrar för individuella kunder respektive företagskunder
Dela försäljning med antal ordrar individuella kunder respektive företagskunder  
Gruppera på region   
Sortera på totalt genomsnittligt ordervärde  
Skapa grupperat stapeldiagram  
Analysera resultatet

In [ ]:
query_vis7 = """
SELECT
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END AS Region,
    SUM(CASE 
            WHEN c.PersonID IS NOT NULL THEN soh.TotalDue 
            ELSE 0 
        END) / NULLIF(COUNT(CASE 
            WHEN c.PersonID IS NOT NULL THEN soh.SalesOrderID 
            ELSE NULL 
        END), 0) AS AvgIndividualOrderValue,
    SUM(CASE 
            WHEN c.StoreID IS NOT NULL THEN soh.TotalDue 
            ELSE 0 
        END) / NULLIF(COUNT(CASE 
            WHEN c.StoreID IS NOT NULL THEN soh.SalesOrderID 
            ELSE NULL 
        END), 0) AS AvgStoreOrderValue,
    SUM(soh.TotalDue) / NULLIF(COUNT(DISTINCT soh.SalesOrderID), 0) AS AvgTotalOrderValue
FROM Sales.SalesTerritory AS st
LEFT JOIN Sales.SalesOrderHeader AS soh
    ON st.TerritoryID = soh.TerritoryID
LEFT JOIN Sales.Customer AS c
    ON soh.CustomerID = c.CustomerID
LEFT JOIN Sales.Store AS s
    ON c.StoreID = s.BusinessEntityID
GROUP BY
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END
ORDER BY AvgTotalOrderValue DESC;
"""

df_vis7 = query_df(query_vis7)

x = np.arange(len(df_vis7))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars_individual = ax.bar(x - width/2, df_vis7["AvgIndividualOrderValue"], width, label="Genomsnittligt ordervärde Individual")
bars_store = ax.bar(x + width/2, df_vis7["AvgStoreOrderValue"], width, label="Genomsnittligt ordervärde Store")

for bar in bars_individual:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 1500,
        f"{height:,.0f}",
        va="center",
        ha="center",
        rotation=45
    )

for bar in bars_store:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 1500,
        f"{int(height)}",
        va="center",
        ha="center",
        rotation=45
    )

ax.set_title("Genomsnittligt ordervärde per region och kundtyp", fontsize=14, fontweight="bold")
ax.set_xlabel("Region", fontsize=12)
ax.set_ylabel("Genomsnittligt ordervärde Individual och Store", fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(df_vis7["Region"], rotation=45)
ax.margins(y=0.15)
ax.legend()

plt.tight_layout()
plt.show()


### Insikter
Analysen visar att företagskunder (Store) genomgående har ett högre genomsnittligt ordervärde än individuella kunder i alla regioner, vilket är rimligt då företagskunder ofta gör större inköp per order. Det som särskilt sticker ut är att US Central, US Northeast och US Southeast även har höga genomsnittliga ordervärden för individuella kunder. Detta indikerar att privatkunder i dessa regioner i större utsträckning lägger större ordrar eller köper dyrare produkter jämfört med andra regioner. En möjlig förklaring kan vara att dessa regioner har ett större utbud av premiumprodukter för privatkunder, en kundgrupp med högre betalningsvilja, eller en starkare cykelkultur där privatpersoner investerar mer i sin utrustning.

För företagskunder är US Southwest den region som har högst genomsnittligt ordervärde. Detta kan tyda på att regionen har fler återförsäljare eller företagskunder som gör färre men större inköp.

Sammanfattningsvis visar analysen att skillnaderna i genomsnittligt ordervärde mellan regioner och kundtyper sannolikt hänger ihop med hur kunderna handlar och vilka typer av produkter som efterfrågas, snarare än enbart hur många kunder som finns i varje region.
 
_____________________________________________________________________________________________________________________________________________________

## Visualisering VG1: Regional försäljningsoptimering
**Affärsfråga:** Vilken region presterar bäst/sämst?

**Tabeller som används:**  
Sales.SalesTerritory  
Sales.SalesOrderHeader  

**Plan:**  
JOINa tabellerna  
Summera total försäljning  
Räkna antal ordrar  
Beräkna genomsnittligt ordervärde  
Gruppera på region   
Sortera på totalt genomsnittligt ordervärde  
Skapa grupperat stapeldiagram  
Analysera resultatet

TotalDue används som mått på försäljning, under antagandet att orderns totalbelopp är det mest relevanta värdet vid analys per region och över tid.

In [ ]:
query_visVG1 = """
SELECT
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END AS Region,
    SUM(soh.TotalDue) AS TotalSales,
    COUNT(DISTINCT soh.SalesOrderID) AS NumberOfOrders,
    SUM(soh.TotalDue) / NULLIF(COUNT(DISTINCT soh.SalesOrderID), 0) AS AvgTotalOrderValue
FROM Sales.SalesTerritory AS st
LEFT JOIN Sales.SalesOrderHeader AS soh
    ON st.TerritoryID = soh.TerritoryID
GROUP BY
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END
ORDER BY AvgTotalOrderValue DESC;
"""

df_visVG1 = query_df(query_visVG1)

x = np.arange(len(df_visVG1))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))

bars_sales = ax.bar(x - width/2, df_visVG1["TotalSales"] / 1_000, width, label="Försäljning (tusental)")
bars_orders = ax.bar(x + width/2, df_visVG1["NumberOfOrders"], width, label="Antal ordrar")

for bar in bars_sales:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 1500,
        f"{height:,.0f}",
        va="center",
        ha="center",
        rotation=45
    )

for bar in bars_orders:
    height = bar.get_height()
    ax.text(
        bar.get_x() + bar.get_width()/2,
        height + 1500,
        f"{height:,.0f}",
        va="center",
        ha="center",
        rotation=45
    )

ax.set_title("Försäljning och antal ordrar per region", fontsize=14, fontweight="bold")
ax.set_xlabel("Region", fontsize=12)
ax.set_ylabel("Försäljning (tusental) / Antal ordrar", fontsize=12)
ax.set_xticks(x)
ax.set_xticklabels(df_visVG1["Region"], rotation=45)
ax.margins(y=0.15)
ax.legend()

plt.tight_layout()
plt.show()

### Insikter
Till skillnad från analysen av total försäljning i visualisering 6 är regionerna här sorterade efter genomsnittligt ordervärde, vilket ger ett annat perspektiv på prestation.  
Det jämför regionernas effektivitet per order, inte deras totala volym.

Regionen US Southwest presterar starkast sett till total försäljning och har även ett högt antal ordrar, vilket indikerar både stor efterfrågan och relativt högt ordervärde.
Detta ligger i linje med resultaten i visualisering 7, där samma region även uppvisar högt genomsnittligt ordervärde för företagskunder.  

Regionen Germany presterar svagast med låg total försäljning trots att antal ordrar inte är lägst, vilket tyder på ett lägre genomsnittligt ordervärde per order.
Australia sticker ut genom att ha flest ordrar men lägst genomsnittligt ordervärde, vilket pekar mot en marknad med många mindre köp och färre stora affärer  

Sammantaget visar analysen att regioners prestation påverkas av olika kombinationer av volym och orderstorlek, och att detta mönster bekräftas av skillnaderna mellan kundsegment i visualisering 7.
__________________________________________________________________________________________________________________________________________

## Visualisering VG2: Regional försäljningsoptimering
**Affärsfråga:** Vilka produktkategorier säljer bäst var?

**Tabeller som används:**  
Sales.SalesTerritory  
Sales.SalesOrderHeader  
Sales.SalesOrderDetail   
Production.Product  
Production.ProductSubcategory  
Production.ProductCategory  

**Plan:**  
JOINa tabellerna  
Summera total försäljning  
Gruppera per region och produktkategori   
Sortera på total försäljning   
Skapa pivotabell  
Skapa heatmap  
Analysera resultatet 

LineTotal används som mått på försäljning på detaljnivå, under antagandet att summan av orderrader bäst speglar försäljningen per produktkategori.

In [ ]:
query_visVG2 = """
SELECT
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END AS Region,
    pc.Name AS Category,
    SUM(sod.LineTotal) AS TotalSales
FROM Sales.SalesTerritory AS st
LEFT JOIN Sales.SalesOrderHeader AS soh
    ON st.TerritoryID = soh.TerritoryID
LEFT JOIN Sales.SalesOrderDetail AS sod
    ON soh.SalesOrderID = sod.SalesOrderID
LEFT JOIN Production.Product AS p
    ON sod.ProductID = p.ProductID
LEFT JOIN Production.ProductSubcategory AS ps
    ON p.ProductSubcategoryID = ps.ProductSubcategoryID
LEFT JOIN Production.ProductCategory AS pc
    ON ps.ProductCategoryID = pc.ProductCategoryID
WHERE sod.LineTotal IS NOT NULL
GROUP BY 
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END,
    pc.Name
ORDER BY TotalSales DESC;
"""

df_visVG2 = query_df(query_visVG2)

piv = pd.pivot_table(
    df_visVG2,
    index="Region",
    columns="Category",
    values="TotalSales",
    aggfunc="sum",
    fill_value=0,
    observed = True
).sort_values(by="Bikes", ascending=False)

display(piv)

piv_plot = piv / 1_000_000

fig, ax = plt.subplots(figsize=(10, 6))

im = ax.imshow(piv_plot.values, aspect="auto", cmap="viridis")

ax.set_xticks(range(len(piv_plot.columns)))
ax.set_xticklabels(piv_plot.columns, rotation=45, ha="right")

ax.set_yticks(range(len(piv_plot.index)))
ax.set_yticklabels(piv_plot.index)

ax.set_title("Heatmap: Försäljning per region och kategori", fontsize=14, fontweight="bold")
ax.set_xlabel("Kategori")
ax.set_ylabel("Region")

cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Total försäljning (miljoner)", rotation=90)

plt.tight_layout()
plt.show()

### Insikter
Heatmapen visar tydligt att produktkategorin Bikes dominerar försäljningen i samtliga regioner. Den starkaste prestationen ses i US Southwest, följt av Canada och US Northwest. Även i europeiska regioner som United Kingdom, France och Germany står Bikes för den största andelen av försäljningen, men på en lägre total nivå jämfört med Nordamerika.

Kategorin Components utgör den näst största försäljningn i de flesta regioner, särskilt i US Southwest, Canada och US Northwest, vilket kan tyda på en större eftermarknad eller fler reparationer och uppgraderingar i dessa områden. Clothing och Accessories har genomgående låg försäljning i samtliga regioner och bidrar endast marginellt till den totala försäljningen.

Sammantaget indikerar analysen att försäljningen är starkt koncentrerad till Bikes, med regionala skillnader främst i volym snarare än i produktmix. Detta antyder att strategier för tillväxt sannolikt bör fokusera på att stärka och vidareutveckla cykelförsäljningen per region, samt undersöka potentialen att öka andelen Components i regioner med hög cykelförsäljning.  _________________________________________________________________________________________________________________________________________________________

## Visualisering VG3: Regional försäljningsoptimering
**Affärsfråga:** Finns säsongsmönster per region?

**Tabeller som används:**  
Sales.SalesTerritory  
Sales.SalesOrderHeader  

**Plan:**  
JOINa tabellerna  
Aggregera försäljning per månad för åren med helårsdata, 2023 och 2024   
Gruppera på region och månad  
Sortera på total försäljning   
Skapa pivotabell  
Skapa heatmap  
Analysera resultatet

TotalDue används som mått på försäljning, under antagandet att orderns totalbelopp är det mest relevanta värdet vid analys per region och över tid.

In [ ]:
query_visVG3 = """
SELECT 
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END AS Region,
    YEAR(soh.OrderDate) AS OrderYear,
    MONTH(soh.OrderDate) AS OrderMonth,
    SUM(soh.TotalDue) AS TotalSales
FROM Sales.SalesTerritory AS st
LEFT JOIN Sales.SalesOrderHeader AS soh
    ON st.TerritoryID = soh.TerritoryID
WHERE soh.OrderDate >= '2023-01-01' AND soh.OrderDate <  '2025-01-01'
GROUP BY 
    CASE
        WHEN st.CountryRegionCode = 'US'
            THEN CONCAT('US ',st.Name)
        ELSE st.Name
    END,
    YEAR(soh.OrderDate),
    MONTH(soh.OrderDate)
ORDER BY Region, OrderMonth ASC;
""" 

df_visVG3 = query_df(query_visVG3)

piv = pd.pivot_table(
    df_visVG3,
    index="Region",
    columns="OrderMonth",
    values="TotalSales",
    aggfunc="mean",
    fill_value=0,
    observed = True
)

piv = piv.loc[piv.sum(axis=1).sort_values(ascending=False).index]

display(piv)

piv_plot = piv / 1_000_000

fig, ax = plt.subplots(figsize=(10, 6))

im = ax.imshow(piv_plot.values, aspect="auto", cmap="cividis")

ax.set_xticks(range(12))
ax.set_xticklabels(piv_plot.columns)
ax.set_yticks(range(len(piv_plot.index)))
ax.set_yticklabels(piv_plot.index)

ax.set_title("Heatmap: Genomsnittlig månadsförsäljning per region (2023-2024)", fontsize=14, fontweight="bold")
ax.set_xlabel("Månad")
ax.set_ylabel("Region")

cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Genomsnittlig försäljning (miljoner)")

plt.tight_layout()
plt.show()


### Insikter
Heatmapen visar tydliga säsongsmönster där flera regioner, särskilt US Southwest och Canada, har högre genomsnittlig försäljning under sommar- och höstmånaderna. Lägre nivåer ses generellt under årets första kvartal, vilket indikerar säsongsvariation i efterfrågan snarare än jämn försäljning över året. I visualisering 3 sågs samma säsongsmönster och föreslogs att detta indikerar att försäljningen följer säsonger då det är bättre förutsättningar för att cykla och den relativst höga försäljningen i december kan vara kopplad till julhandeln. __________________________________________________________________________________________________________________________________________________

## Sammanfattande insikter regional försäljningsoptimering
Analysen visar att US Southwest är den starkast presterande regionen sett till både total försäljning och genomsnittligt ordervärde. Regionen kombinerar hög försäljningsvolym med relativt höga ordervärden, vilket indikerar en mogen och välfungerande marknad. Canada och US Northwest uppvisar också stark försäljning, främst drivet av höga volymer inom kategorin Bikes, men med något lägre genomsnittligt ordervärde jämfört med US Southwest. Detta tyder på god efterfrågan men med potential att öka värdet per order.

I flera regioner, särskilt US Central, US Northeast och US Southeast, är det intressant att även individuella kunder har relativt höga genomsnittliga ordervärden. Detta indikerar att privatkunder i dessa regioner i högre grad köper dyrare produkter eller större paket snarare än enstaka lågprisartiklar.

Heatmap-analysen för produktkategorier visar att Bikes dominerar försäljningen i samtliga regioner, medan Components har näst störst försäljning. Clothing och Accessories bidrar endast marginellt till den totala försäljningen.

Säsongsanalysen visar tydliga mönster, där försäljningen är högre runt sommarmånaderna, särskilt i Nordamerika. Detta indikerar att efterfrågan i stor utsträckning är säsongsberoende och kopplad till användning snarare än jämn konsumtion över året.

## Rekommendationer för förbättring
- Fokusera fortsatt på cykelförsäljning, särskilt i regioner med redan hög försäljning som US Southwest, Canada och US Northwest.
- Öka andelen Components i starka cykelregioner, exempelvis genom paketlösningar, serviceerbjudanden eller kampanjer kopplade till cykelköp.
- Utforska varför individuella kunder i vissa USA-regioner har höga ordervärden, och om detta beteende kan skalas till andra regioner genom sortiment, prissättning eller marknadsföring.
- Anpassa marknadsföring och lager efter säsong, med ökat fokus runt sommarperioden då efterfrågan är som högst.
- Identifiera lågpresterande regioner som Germany och Australia för att utvärdera om orsaken är begränsat sortiment, lägre efterfrågan eller mindre utvecklade försäljningskanaler.